In [ ]:
import sys

from IPython.display import Image, display
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

: 

In [ ]:
%cd '/content/drive/MyDrive/Inteligencia Artificial/IA - Clases de Práctica/ContenidosPorTemas/ActividadesPracticas'

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Segmentación de clientes 👩🤠🧑

El siguiente conjunto de datos consiste en información sobre el comportamiento de compra de 2,000 individuos de una determinada área al entrar en una tienda física de bienes de consumo rápido. Todos los datos se han recopilado a través de las tarjetas que utilizan al momento de pagar. Los datos han sido preprocesados y no hay valores faltantes. Además, el volumen del conjunto de datos se ha limitado y anonimizado para proteger la privacidad de los clientes.

Columnas del conjunto de datos:

- **Sex**: género.
        0: masculino,
        1: femenino.
- **Marital status**: Estado civil de un cliente.
        0: soltero,
        1: no soltero (divorciado/separado/casado/viudo).
- **Age**: Edad del cliente en años. Valor mínimo: 18, Valor máximo: 78.
- **Education**: Nivel educativo del cliente.
        0: otro/desconocido,
        1: escuela secundaria,
        2: universidad,
        3: posgrado.
- **Income**: Ingreso anual autoreportado en dólares estadounidenses del cliente. Valor mínimo: 35,832, Valor máximo: 309,364.
- **Occupation**: Categoría de la ocupación del cliente.
        0: desempleado/no calificado,
        1: empleado calificado/funcionario,
        2: administración/autónomo/empleado altamente calificado/oficial.
- **Settlement size**: Tamaño de la ciudad donde vive el cliente.
        0: ciudad pequeña,
        1: ciudad mediana,
        2: gran ciudad.

El problema es que tenemos muchas variables (edad, ingresos, estado civil, etc.), y no es fácil trabajar con todas a la vez. Vamos a usar una técnica muy útil: Análisis de Componentes Principales (PCA, por sus siglas en inglés: Principal Component Analysis).

¿Qué hace PCA?

- Reduce la dimensionalidad de los datos, es decir, transforma muchas variables en unas pocas nuevas llamadas componentes principales.
- Estas nuevas variables son combinaciones lineales de las originales y están ordenadas de forma que:

    - La primera componente explica la mayor parte de la variabilidad en los datos.
    - La segunda componente explica la mayor parte de la variabilidad que queda, y así sucesivamente.

De esta manera podemos simplificar los datos, visualizarlos en 2D o 3D y usarlos como entrada para algoritmos de clustering sin perder demasiada información.

```
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print("Varianza explicada por cada componente:", pca.explained_variance_ratio_)

```


### Actividad

Con los datos reducidos (X_pca) aplique los tres métodos distintos de clustering:

K-Means: elegir el valor de K usando método del codo y/o de silueta.
DBSCAN: detectar densidades y posibles outliers.
Jerárquico (Agglomerative): formar clusters basados en similitud.

La **segmentación de clientes** (**customer segmentation**) es una estrategia y técnica en marketing y análisis de datos que implica dividir una base de clientes en grupos más pequeños y homogéneos según características y comportamientos similares. El objetivo principal de la segmentación de clientes es comprender mejor a los diferentes grupos de clientes para poder adaptar estrategias y enfoques de marketing específicos a cada segmento.

Luego de probar los distintos métodos, seleccionar uno y agrupar en el número de clusters seleccionado. Con las etiquetas asignadas por el método, filtrar el dataset original y describir en términos de las características originales cada uno de los grupos.


In [ ]:
# Cargo los datos
df = pd.read_csv("1_datos/customer_data.csv", delimiter=",", header=0)
datum = pd.DataFrame(df)
datum.head(10)

In [ ]:
import seaborn as sns
# Analisis de datos: Basico

datum.info()

plt.figure(figsize=(10, 6))
sns.countplot(data=datum, x="Sex", hue="Education")
plt.title("Distribución de Educación por Sexo")
plt.show()

In [ ]:
# PairGrid
g = sns.PairGrid(datum[['Age', 'Income','Education']], hue='Education')
g.map_upper(sns.scatterplot)
g.map_lower(sns.scatterplot)
g.map_diag(sns.histplot)
g.add_legend()
plt.suptitle('PairGrid Personalizado', y=1.02)
plt.show()

In [ ]:
# Agrego mas visualizaciones de variables

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Mix de Gráficos con Seaborn', fontsize=16, fontweight='bold')
    
# Gráfico 1: Violin plot
sns.violinplot(data=datum, x='Sex', y='Age', ax=axes[0, 0])
axes[0, 0].set_title('Violin Plot: Edad por Categoría')
    
# Gráfico 2: Heatmap de correlación
corr_data = datum[['x', 'y', 'Settlement size', 'Income']].corr()
sns.heatmap(corr_data, annot=True, cmap='coolwarm', center=0, ax=axes[0, 1])
axes[0, 1].set_title('Matriz de Correlación')
    
# Gráfico 3: Strip plot
sns.stripplot(data=datum, x='Marital status', y='Occupation', hue='Occupation', ax=axes[0, 2])
axes[0, 2].set_title('Strip Plot: Salario por Grupo')
axes[0, 2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
# Gráfico 4: Regression plot
sns.regplot(data=datum, x='Age', y='Income', ax=axes[1, 0])
axes[1, 0].set_title('Regresión: Edad vs Salario')
    
# Gráfico 5: Bar plot
sns.barplot(data=datum, x='categoria', y='salario', hue='grupo', ax=axes[1, 1])
axes[1, 1].set_title('Salario Promedio por Categoría y Grupo')
    
# Gráfico 6: KDE plot
sns.kdeplot(data=datos, x='edad', y='salario', ax=axes[1, 2])
axes[1, 2].set_title('Densidad: Edad vs Salario')
    
plt.tight_layout()
plt.show()

In [ ]:
# Escalamos los datos

# 1. Seleccionar las columnas que querés escalar
cols = ["ID", "ingresos"]

# 2. Crear el escalador
scaler = StandardScaler()

# 3. Ajustar y transformar
X_scaled = scaler.fit_transform(df[cols])

# 4. Pasar el resultado a DataFrame con los mismos nombres de columnas
df_scaled = pd.DataFrame(X_scaled, columns=cols)

print(df_scaled)

In [ ]:
from sklearn.decomposition import PCA
# Aplico PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(datum)

print("Varianza explicada por cada componente:", pca.explained_variance_ratio_)
